In [1]:
class Text:
    def __init__(self):
        self.colors = {
            'black'    : '\033[30m',
            'red'      : '\033[31m',
            'green'    : '\033[32m',
            'yello'    : '\033[33m',
            'blue'     : '\033[34m',
            'purple'   : '\033[35m',
            'cyan'     : '\033[36m',
            'white'    : '\033[37m',
        }
        self.bold      = '\033[1m'
        self.underline = '\033[4m'
        self.invisible = '\033[08m'
        self.reverse   = '\033[07m'
        self.end       = '\033[0m'
    
    def text(self, text, color='black', bold=False, underline=False, invisible=False, reverse=False):
        result = ''
        if color in self.colors: result += self.colors[color]
        if bold: result += self.bold
        if underline: result += self.underline
        if invisible: result += self.invisible
        if reverse: result += self.reverse
        
        result += text
        result += self.end
        
        return result

In [9]:
# coding: utf-8
from IPython.display import clear_output
import time
import random

class Reverse:
    def __init__(self):
        self.board = [['_', '_', '_', '_', '_', '_', '_', '_'],
                      ['_', '_', '_', '_', '_', '_', '_', '_'],
                      ['_', '_', '_', '_', '_', '_', '_', '_'],
                      ['_', '_', '_', 'W', 'B', '_', '_', '_'],
                      ['_', '_', '_', 'B', 'W', '_', '_', '_'],
                      ['_', '_', '_', '_', '_', '_', '_', '_'],
                      ['_', '_', '_', '_', '_', '_', '_', '_'],
                      ['_', '_', '_', '_', '_', '_', '_', '_']]
        self.next_step = 'B'
        self.pass_count = 0
        self.list_enable_pos = []
        return

    def game_play(self):
        while True:
            self.list_enable_pos = self.get_pos_enable()
            if [] != self.list_enable_pos:
                # 置く場所がある場合
                text = Text()
                clear_output()
                num_black = sum([b.count('B') for b in self.board])
                num_white = sum([b.count('W') for b in self.board])
                next_step = "Black" if self.next_step == 'B' else "White"
                prev_step = "White" if self.next_step == 'B' else "Black"

                if self.pass_count > 0:
                    print(f"There is no position where {prev_step} can put into.")
                
                print(f"Next Step is {text.text(next_step, color='red', bold=True)}")
                self.draw_board()
                pos = self.get_input()

                # 裏返す
                pos_reverse = self.get_pos_reverse(pos)                    
                for pos in pos_reverse:
                    self.board[pos[0]][pos[1]] = self.next_step

            else:
                # 置く場所がない場合
                self.draw_board()
                self.pass_count += 1

            self.next_step = 'B' if self.next_step == 'W' else 'W'
            # 終了判断　連続パス or 全部の場所に石が置かれる
            if self.pass_count >= 2 or 0 >= sum([b.count('_') for b in self.board]):
                text = Text()
                clear_output()
                
                num_black = sum([b.count('B') for b in self.board])
                num_white = sum([b.count('W') for b in self.board])
                
                if num_black > num_white:
                    print(text.text("Winner: Black", color="red", bold=True))
                elif num_black < num_white:
                    print(text.text("Winner: White", color="red", bold=True))
                else:
                    print(text.text("Draw", color="red", bold=True))
                
                self.draw_board()
                return
    
    def draw_board(self):
        text = Text()      
        num_black = sum([b.count('B') for b in self.board])
        num_white = sum([b.count('W') for b in self.board])
        next_step = "Black" if self.next_step == 'B' else "White"
        prev_step = "White" if self.next_step == 'B' else "Black"

        print("=" * 30)
        print(f"number of black: {num_black}")
        print(f"number of white: {num_white}")
        
        print("  a b  c  d  e f  g  h")
        for index, raw in enumerate(self.board):
            print(str(index+1) + " " + "　".join(raw))
        print("=" * 30)
        print()
        return

    # 設定する位置のリストを返す
    def get_pos_reverse(self, pos):
        lines = [[0, 1],
                 [0, -1],
                 [1, 0],
                 [1, 1],
                 [1, -1],
                 [-1, 0],
                 [-1, 1],
                 [-1, -1]]

        pos_reverse_all = []
        for line in lines:
            isCheck = False # 裏返す対象の有無
            next_pos = pos
            for i in range(len(self.board)):
                next_pos = [(next_pos[0] + line[0]),(next_pos[1] + line[1])]
                if ( ( next_pos[0] < 0 ) or ( len(self.board) <= next_pos[0] ) or
                     ( next_pos[1] < 0 ) or ( len(self.board) <= next_pos[1] ) ):
                    # 端までチェック完了、裏返す対象がない
                    break
                else:
                    if ( '_' == self.board[next_pos[0]][next_pos[1]] or 'x' == self.board[next_pos[0]][next_pos[1]]):
                        # 裏返す対象がない
                        break
                    elif self.next_step != self.board[next_pos[0]][next_pos[1]]:
                        isCheck = True
                    else:
                        if True == isCheck:
                            isCheck = True
                            pos_check = [(next_pos[0] - line[0]),(next_pos[1] - line[1])]
                            while pos != pos_check:
                                pos_reverse_all += [pos_check]
                                pos_check = [(pos_check[0] - line[0]),(pos_check[1] - line[1])]
                        else:
                            # 裏返す対象がない
                            break
        
        if [] != pos_reverse_all:
            pos_reverse_all += [pos]
                
        return pos_reverse_all
    
    # 入力可能箇所を返す
    def get_pos_enable(self):
        result = []
        for col in range(len(self.board)):
            for raw in range(len(self.board[0])):
                if '_' == self.board[col][raw]:
                    pos = [col,raw]
                    if [] != self.get_pos_reverse(pos):
                        result += [pos]
        return result

    # 入力を取得する関数
    def get_input(self):
        raw = {0: '1',1: '2', 2: '3', 3: '4', 4: '5', 5: '6', 6: '7', 7: '8'}
        col = {0: 'a',1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h'}
        
        while True:
            print("There are position where you can move.")
            for idx,pos in enumerate(self.list_enable_pos):
                print(f"[{idx}] {col[pos[1]]}{raw[pos[0]]}")
                
            try:
                move = input("Please input next move.")
                move = int(move) 
                if move < len(self.list_enable_pos):
                    return self.list_enable_pos[move]
                else:
                    print("[Error] Invalid Input")
            except:
                print("[Error] Invalid Input")

        return self.list_enable_pos[random.randrange(0, len(self.list_enable_pos))]


In [11]:
from IPython.display import clear_output

reverse = Reverse()
reverse.game_play()
    

Winner: White
number of black: 9
number of white: 14
  a b  c  d  e f  g  h
1 W　W　W　W　W　W　W　_
2 W　W　W　W　W　W　_　_
3 W　B　B　B　_　_　_　_
4 _　_　B　B　B　_　_　_
5 _　_　_　B　B　B　_　_
6 _　_　_　_　_　_　_　_
7 _　_　_　_　_　_　_　_
8 _　_　_　_　_　_　_　_

